In [ ]:
import pandas as pd

In [3]:
data = pd.read_csv('data.csv')

In [4]:
# Drop rows with pitch type Other or NaN
data = data[data['PITCH_NAME'] != "Other"]
data = data.dropna(subset=['PITCH_TYPE'])

#### Mapping to Pitch Category

In [5]:
pitch_mapping = {
    '4-Seam Fastball': ['FB'],
    'Slider': ['BB'],
    'Sinker': ['FB'], # SHOULD THIS BE BREAKING TOO
    'Knuckle Curve': ['BB'],
    'Curveball': ['BB'],
    'Changeup': ['OS'],
    'Cutter': ['FB'],
    'Sweeper': ['BB'],
    'Split-Finger': ['OS'],
    'Slurve': ['BB', 'OS'],
    'Pitch Out': ['OS'],
    'Slow Curve': ['BB', 'OS'],
    'Eephus': ['OS'],
    'Knuckleball': ['BB', 'OS'], # should this just be OS?? since knuckle curve is BB
    'Forkball': ['OS', 'BB'],
    'Screwball': ['BB'],
}

data['PITCH_GROUP'] = data['PITCH_NAME'].map(pitch_mapping)

In [6]:
cols_to_keep = ['PITCH_TYPE', 'PITCH_NAME', 'PLAYER_NAME', 'BATTER_ID','GAME_PK', 'GAME_YEAR', 'GAME_DATE','OUTS_WHEN_UP', 'BALLS', 'STRIKES', 'ON_1B', 'ON_2B', 'ON_3B', 'EVENTS', 'DESCRIPTION', 'TYPE', 'ZONE', 'PLATE_X', 'PLATE_Z',
'BB_TYPE', 'HIT_DISTANCE_SC','LAUNCH_SPEED', 'LAUNCH_ANGLE', 'ESTIMATED_BA_USING_SPEEDANGLE', 'ESTIMATED_WOBA_USING_SPEEDANGLE', 'WOBA_VALUE', 'BABIP_VALUE', 'ISO_VALUE', 'LAUNCH_SPEED_ANGLE',  'PITCH_GROUP']       
data = data[cols_to_keep]
data.keys()

Index(['PITCH_TYPE', 'PITCH_NAME', 'PLAYER_NAME', 'BATTER_ID', 'GAME_PK',
       'GAME_YEAR', 'GAME_DATE', 'OUTS_WHEN_UP', 'BALLS', 'STRIKES', 'ON_1B',
       'ON_2B', 'ON_3B', 'EVENTS', 'DESCRIPTION', 'TYPE', 'ZONE', 'PLATE_X',
       'PLATE_Z', 'BB_TYPE', 'HIT_DISTANCE_SC', 'LAUNCH_SPEED', 'LAUNCH_ANGLE',
       'ESTIMATED_BA_USING_SPEEDANGLE', 'ESTIMATED_WOBA_USING_SPEEDANGLE',
       'WOBA_VALUE', 'BABIP_VALUE', 'ISO_VALUE', 'LAUNCH_SPEED_ANGLE',
       'PITCH_GROUP'],
      dtype='object')

### Calculate Proportions for 2021-2023

In [7]:
def calculate_pitch_type_percentages(df, year):
    # Explode the PITCH_GROUP column to separate multiple groups
    df_exploded = df.explode('PITCH_GROUP')
    
    # Count occurrences of each pitch group for each batter
    pitch_group_counts = pd.crosstab(df_exploded['BATTER_ID'], df_exploded['PITCH_GROUP'])
    
    # Calculate total pitches for each batter
    pitch_group_counts['total_pitches'] = pitch_group_counts.sum(axis=1)
    
    # Calculate percentages for each pitch group
    for group in ['FB', 'BB', 'OS']:
        pitch_group_counts[f'%_{group}_{year}'] = (pitch_group_counts.get(group, 0) / pitch_group_counts['total_pitches']) * 100
    
    # Keep only the percentage columns and BATTER_ID
    return pitch_group_counts[[f'%_FB_{year}', f'%_BB_{year}', f'%_OS_{year}']].reset_index()

# Filter data by year
data_2021 = data[data['GAME_YEAR'] == 2021]
data_2022 = data[data['GAME_YEAR'] == 2022]
data_2023 = data[data['GAME_YEAR'] == 2023]  # New filter for 2023

# Calculate percentages for 2021, 2022, and 2023
percentages_2021 = calculate_pitch_type_percentages(data_2021, 2021)
percentages_2022 = calculate_pitch_type_percentages(data_2022, 2022)
percentages_2023 = calculate_pitch_type_percentages(data_2023, 2023)  # New calculation for 2023

# Merge the three years together
percents_by_year = pd.merge(percentages_2021, percentages_2022, on='BATTER_ID', how='outer', suffixes=('_2021', '_2022'))
percents_by_year = pd.merge(percents_by_year, percentages_2023, on='BATTER_ID', how='outer')

# Display the final dataframe
print(percents_by_year.head())


PITCH_GROUP  BATTER_ID  %_FB_2021  %_BB_2021  %_OS_2021  %_FB_2022  %_BB_2022  \
0               444482  56.397875  26.798648  16.803477  53.271538  27.917121   
1               453568  60.883721  23.209302  15.906977  61.600762  23.963792   
2               456781  59.502262  30.693816   9.803922  57.845632  33.672604   
3               457705  54.298056  34.384449  11.317495  58.045493  31.718618   
4               457759  58.454693  31.674757   9.870550  62.995800  27.064862   

PITCH_GROUP  %_OS_2022  %_FB_2023  %_BB_2023  %_OS_2023  
0            18.811341  54.278250  26.649249  19.072502  
1            14.435445  61.573770  23.672131  14.754098  
2             8.481764  57.750420  30.945719  11.303861  
3            10.235889  55.500496  32.210109  12.289395  
4             9.939337  62.193309  28.736059   9.070632  


#### Sort Original Data into Separate Dataframes by Pitch Group and Year

In [9]:
# Explode the pitch group to handle multiple pitch group values in a list
data_exploded = data.explode('PITCH_GROUP')

# Function to filter data by year and pitch group
def filter_by_pitch_group_and_year(df, pitch_group, year):
    return df[(df['PITCH_GROUP'] == pitch_group) & (df['GAME_YEAR'] == year)]

# Create six separate dataframes for each pitch group and year
OS_2021 = filter_by_pitch_group_and_year(data_exploded, 'OS', 2021)
BB_2021 = filter_by_pitch_group_and_year(data_exploded, 'BB', 2021)
FB_2021 = filter_by_pitch_group_and_year(data_exploded, 'FB', 2021)

OS_2022 = filter_by_pitch_group_and_year(data_exploded, 'OS', 2022)
BB_2022 = filter_by_pitch_group_and_year(data_exploded, 'BB', 2022)
FB_2022 = filter_by_pitch_group_and_year(data_exploded, 'FB', 2022)

OS_2023 = filter_by_pitch_group_and_year(data_exploded, 'OS', 2023)
BB_2023 = filter_by_pitch_group_and_year(data_exploded, 'BB', 2023)
FB_2023 = filter_by_pitch_group_and_year(data_exploded, 'FB', 2023)

#### Create Summary Stats by Year and Pitch Group for Each Player to Train Model

In [17]:
def calculate_summary_statistics(df):
    stats = df.groupby('BATTER_ID').agg(
        PERC_FOUL=('DESCRIPTION', lambda x: (x == 'foul').mean() * 100),
        PERC_SWINGING_STRIKE=('DESCRIPTION', lambda x: (x.isin(['swinging_strike', 'swinging_strike_blocked'])).mean() * 100),
        PERC_HIT_INTO_PLAY=('DESCRIPTION', lambda x: (x == 'hit_into_play').mean() * 100),
        AVG_WOBA_HIT_INTO_PLAY=('WOBA_VALUE', lambda x: x[df['DESCRIPTION'] == 'hit_into_play'].mean()),
        PERC_EXTRA_BASE_HITS=('EVENTS', lambda x: (x.isin(['double', 'triple', 'home_run'])).mean() * 100),
        AVG_LAUNCH_SPEED=('LAUNCH_SPEED', 'mean'),
        AVG_LAUNCH_ANGLE=('LAUNCH_ANGLE', 'mean'),
        PERC_PRODUCTIVE_CONTACT=('EVENTS', lambda x: (x.isin(['sac_fly', 'double', 'triple', 'single', 'home_run', 'sac_bunt'])).mean() * 100),
        AVG_BABIP=('BABIP_VALUE', 'mean'),
        AVG_ISO=('ISO_VALUE', 'mean')
    ).reset_index()
    
    return stats

PLAYER_STATS_OS_2021 = calculate_summary_statistics(OS_2021)
PLAYER_STATS_BB_2021 = calculate_summary_statistics(BB_2021)
PLAYER_STATS_FB_2021 = calculate_summary_statistics(FB_2021)

PLAYER_STATS_OS_2022 = calculate_summary_statistics(OS_2022)
PLAYER_STATS_BB_2022 = calculate_summary_statistics(BB_2022)
PLAYER_STATS_FB_2022 = calculate_summary_statistics(FB_2022)

PLAYER_STATS_OS_2023 = calculate_summary_statistics(OS_2023)
PLAYER_STATS_BB_2023 = calculate_summary_statistics(BB_2023)
PLAYER_STATS_FB_2023 = calculate_summary_statistics(FB_2023)

### Adding More Descriptive ID Column for Merging Later

In [19]:
# Dictionary to store the player stats dataframes with the updated names
player_stats_dataframes = {
    'PLAYER_STATS_OS_2021': PLAYER_STATS_OS_2021,
    'PLAYER_STATS_BB_2021': PLAYER_STATS_BB_2021,
    'PLAYER_STATS_FB_2021': PLAYER_STATS_FB_2021,
    'PLAYER_STATS_OS_2022': PLAYER_STATS_OS_2022,
    'PLAYER_STATS_BB_2022': PLAYER_STATS_BB_2022,
    'PLAYER_STATS_FB_2022': PLAYER_STATS_FB_2022,
    'PLAYER_STATS_OS_2023': PLAYER_STATS_OS_2023,
    'PLAYER_STATS_BB_2023': PLAYER_STATS_BB_2023,
    'PLAYER_STATS_FB_2023': PLAYER_STATS_FB_2023
}

# Loop through each player stats dataframe and add the 'ID' column
for key, df in player_stats_dataframes.items():
    pitch_group, year = key.split('_')[2:]  # Extract pitch group and year from the dataframe name
    df['ID'] = df.apply(lambda row: f"{pitch_group}_{year}_{row['BATTER_ID']}", axis=1)

### Adding Columns for Percent of Pitch Type Received during Current and Following Year for Training Model

In [23]:
# Add PERC_RECEIVED and PERC_RECEIVED_FOLLOWING_YEAR columns to each player stats dataframe
for df_name, df in player_stats_dataframes.items():
    pitch_group, year = df_name.split('_')[2:]  # Extract pitch group (OS, BB, FB) and year (2021, 2022, 2023)
    year = int(year)  # Convert year to an integer

    # Determine the correct percentage columns for the current year and the following year
    perc_col_current_year = f'%_{pitch_group}_{year}'
    perc_col_next_year = f'%_{pitch_group}_{year + 1}' if year != 2023 else None

    # Map the current year's percentage received
    df['PERC_RECEIVED'] = df['BATTER_ID'].map(percents_by_year.set_index('BATTER_ID')[perc_col_current_year])

    # Map the following year's percentage received (for 2021 and 2022 only)
    if year != 2023:
        df['PERC_RECEIVED_FOLLOWING_YEAR'] = df['BATTER_ID'].map(percents_by_year.set_index('BATTER_ID')[perc_col_next_year])
    else:
        df['PERC_RECEIVED_FOLLOWING_YEAR'] = float('nan')  # Set NaN for 2023 since no following year exists

# Example of how to access PLAYER_STATS_OS_2021 with the new columns
print(PLAYER_STATS_OS_2021.head())


   BATTER_ID  PERC_FOUL  PERC_SWINGING_STRIKE  PERC_HIT_INTO_PLAY  \
0     444482  11.494253             14.367816           22.988506   
1     453568  13.742690             12.280702           26.608187   
2     456781  18.461538             12.307692           24.615385   
3     457705  12.977099             15.648855           16.412214   
4     457759  14.754098              7.786885           21.721311   

   AVG_WOBA_HIT_INTO_PLAY  PERC_EXTRA_BASE_HITS  AVG_LAUNCH_SPEED  \
0                0.370000              2.873563         81.124138   
1                0.312088              2.631579         83.044118   
2                0.370312              2.307692         79.825926   
3                0.255814              1.526718         81.956944   
4                0.394340              1.639344         83.843373   

   AVG_LAUNCH_ANGLE  PERC_PRODUCTIVE_CONTACT  AVG_BABIP   AVG_ISO  \
0          2.672414                 6.896552   0.221154  0.144231   
1          6.448529             

### Remove Accidental Duplicate Column and Export JSONs

In [ ]:
# Remove duplicate BATTER_ID columns from each PLAYER_STATS dataframe
for df_name, df in player_stats_dataframes.items():
    if 'BATTER_ID' in df.columns:
        # Drop duplicate BATTER_ID columns, keeping the first occurrence
        df = df.loc[:, ~df.columns.duplicated()]
        # Update the DataFrame in the dictionary
        player_stats_dataframes[df_name] = df

# Export each dataframe to a JSON file again
for df_name, df in player_stats_dataframes.items():
    df.to_json(f'{df_name}.json', orient='records', lines=True)

In [12]:
print(len(PLAYER_STATS_OS_2023))
print(len(PLAYER_STATS_OS_2022))
print(len(PLAYER_STATS_OS_2021))

310
269
220


## NaN Cleanup: if perc_hit_into_play = 0.0, remove nans in columns affected and fill with 0.0

In [1]:
# columns to check for nans
nan_columns = ['AVG_ISO', 'AVG_BABIP', 'AVG_LAUNCH_SPEED', 'AVG_LAUNCH_ANGLE', 'AVG_WOBA_HIT_INTO_PLAY']

# Function to process each dataframe
def process_dataframe(df):
    # Replace NaNs in the specified columns with 0 if 'PERC_HIT_INTO_PLAY' is 0.0
    for col in nan_columns:
        df.loc[(df['PERC_HIT_INTO_PLAY'] == 0.0) & (df[col].isna()), col] = 0.0
    return df

# List of JSON file paths
json_files = [
    'PLAYER_STATS_FB_2021.json', 'PLAYER_STATS_FB_2022.json', 'PLAYER_STATS_FB_2023.json',
    'PLAYER_STATS_BB_2021.json', 'PLAYER_STATS_BB_2022.json', 'PLAYER_STATS_BB_2023.json',
    'PLAYER_STATS_OS_2021.json', 'PLAYER_STATS_OS_2022.json', 'PLAYER_STATS_OS_2023.json'
]

# Iterate through the JSON files and process each one
for json_file in json_files:
    # Load the dataframe from JSON with lines=True
    df = pd.read_json(json_file, lines=True)

    # Process the dataframe (deal with NaNs)
    df_processed = process_dataframe(df)

    # Re-export the processed dataframe back to JSON with the same filename (lines=True to preserve structure)
    df_processed.to_json(json_file, orient='records', lines=True)

    print(f"Processed and saved: {json_file}")


Processed and saved: PLAYER_STATS_FB_2021.json
Processed and saved: PLAYER_STATS_FB_2022.json
Processed and saved: PLAYER_STATS_FB_2023.json
Processed and saved: PLAYER_STATS_BB_2021.json
Processed and saved: PLAYER_STATS_BB_2022.json
Processed and saved: PLAYER_STATS_BB_2023.json
Processed and saved: PLAYER_STATS_OS_2021.json
Processed and saved: PLAYER_STATS_OS_2022.json
Processed and saved: PLAYER_STATS_OS_2023.json


/var/folders/t8/nqklvy6x565bg7rjp1gqrcl40000gn/T/ipykernel_18021/4030030585.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [5]:
df = pd.read_json('PLAYER_STATS.json', lines=True)

# Process the dataframe (deal with NaNs)
df_processed = process_dataframe(df)

# Re-export the processed dataframe back to JSON with the same filename (lines=True to preserve structure)
df_processed.to_json('PLAYER_STATS.json', orient='records', lines=True)

#### Affirm NaNs have been removed


In [6]:
df = pd.read_json('PLAYER_STATS.json', lines=True)
nan_rows = df[(df.drop(columns=['PERC_RECEIVED_FOLLOWING_YEAR']).isna().any(axis=1))]
print(nan_rows)

Empty DataFrame
Columns: [BATTER_ID, ID, PERC_RECEIVED, PERC_RECEIVED_FOLLOWING_YEAR, PERC_FOUL, PERC_SWINGING_STRIKE, PERC_HIT_INTO_PLAY, AVG_WOBA_HIT_INTO_PLAY, PERC_EXTRA_BASE_HITS, AVG_LAUNCH_SPEED, AVG_LAUNCH_ANGLE, PERC_PRODUCTIVE_CONTACT, AVG_BABIP, AVG_ISO]
Index: []


In [2]:
# Load player stats dataframes for each pitch type and year from JSON files
PLAYER_STATS_FB_2023 = pd.read_json('PLAYER_STATS_FB_2023.json', lines=True)
PLAYER_STATS_BB_2023 = pd.read_json('PLAYER_STATS_BB_2023.json', lines=True)
PLAYER_STATS_OS_2023 = pd.read_json('PLAYER_STATS_OS_2023.json', lines=True)

PLAYER_STATS_FB_2022 = pd.read_json('PLAYER_STATS_FB_2022.json', lines=True)
PLAYER_STATS_BB_2022 = pd.read_json('PLAYER_STATS_BB_2022.json', lines=True)
PLAYER_STATS_OS_2022 = pd.read_json('PLAYER_STATS_OS_2022.json', lines=True)

PLAYER_STATS_FB_2021 = pd.read_json('PLAYER_STATS_FB_2021.json', lines=True)
PLAYER_STATS_BB_2021 = pd.read_json('PLAYER_STATS_BB_2021.json', lines=True)
PLAYER_STATS_OS_2021 = pd.read_json('PLAYER_STATS_OS_2021.json', lines=True)


player_stats = {
    'FB': [PLAYER_STATS_FB_2023, PLAYER_STATS_FB_2022, PLAYER_STATS_FB_2021],
    'BB': [PLAYER_STATS_BB_2023, PLAYER_STATS_BB_2022, PLAYER_STATS_BB_2021],
    'OS': [PLAYER_STATS_OS_2023, PLAYER_STATS_OS_2022, PLAYER_STATS_OS_2021]
}

In [3]:
# Iterate through the dictionary and check for NaN elements in each dataframe, excluding PERC_RECEIVED_FOLLOWING_YEAR
for pitch_type, df_list in player_stats.items():
    print(f"\nChecking for NaN elements in {pitch_type} dataframes:")
    for i, df in enumerate(df_list):
        # Get the year based on the position in the list (assuming the years are 2023, 2022, 2021)
        year = 2023 - i
        # Filter for NaN values, excluding PERC_RECEIVED_FOLLOWING_YEAR
        nan_rows = df[(df.drop(columns=['PERC_RECEIVED_FOLLOWING_YEAR']).isna().any(axis=1))]
        
        if nan_rows.empty:
            print(f"No NaN values found in {pitch_type} for {year}.")
        else:
            print(f"\nNaN values in {pitch_type} for {year} (excluding PERC_RECEIVED_FOLLOWING_YEAR):")
            print(nan_rows)


Checking for NaN elements in FB dataframes:
No NaN values found in FB for 2023.
No NaN values found in FB for 2022.
No NaN values found in FB for 2021.

Checking for NaN elements in BB dataframes:
No NaN values found in BB for 2023.
No NaN values found in BB for 2022.
No NaN values found in BB for 2021.

Checking for NaN elements in OS dataframes:
No NaN values found in OS for 2023.
No NaN values found in OS for 2022.
No NaN values found in OS for 2021.


**checking num of players each year**

In [13]:
# Filter data by year
data_2021 = data[data['GAME_YEAR'] == 2021]
data_2022 = data[data['GAME_YEAR'] == 2022]
data_2023 = data[data['GAME_YEAR'] == 2023]

# Count unique BATTER_IDs in each year
unique_players_2021 = data_2021['BATTER_ID'].nunique()
unique_players_2022 = data_2022['BATTER_ID'].nunique()
unique_players_2023 = data_2023['BATTER_ID'].nunique()

# Print the results
print(f"Number of unique players in 2021: {unique_players_2021}")
print(f"Number of unique players in 2022: {unique_players_2022}")
print(f"Number of unique players in 2023: {unique_players_2023}")


Number of unique players in 2021: 222
Number of unique players in 2022: 270
Number of unique players in 2023: 310
